In [2]:
import pandas as pd
import numpy as np
import re
from nltk.stem.snowball import RussianStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import pickle

In [3]:
data = pd.read_csv('news.csv')

In [4]:
data = data.dropna(subset=['rubric'])

In [5]:
for c in data['rubric'].unique():
    print(c, sum(data['rubric'] == c))

Экономика 420
Спорт 494
Из жизни 197
Интернет и СМИ 211
Культура 206
Дом 123
Бывший СССР 394
69-я параллель 27
Мир 718
Наука и техника 208
Путешествия 183
Россия 908
Нацпроекты 53
Силовые структуры 220
Ценности 141


In [6]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(np.array(data['rubric']).reshape(-1, 1)).astype('int').toarray()
column_to_category = list(ohe.categories_[0])

In [7]:
text = np.array(data['text'])

stem_rus = RussianStemmer(False)

for i in range(len(text)):
    out = ''
    for word in re.sub('[^{0-9а-яА-Я}]', ' ', text[i]).lower().split(' '):
        out += stem_rus.stem(word) + ' '
    text[i] = out

In [8]:
basic_vectorizer = CountVectorizer(min_df=4)
text_train = text[:4000]
text_test = text[4000:]
X_train = basic_vectorizer.fit_transform(text_train)
X_test = basic_vectorizer.transform(text_test)
y_train = y[:4000, -1]
y_test = y[4000:, -1]
# В исходном датасете статьи были перемешаны, поэтому можно разделить даже так

In [9]:
from ensembles import RandomForestMSE

In [10]:
from sklearn.metrics import mean_squared_error

grid = {
    'n_estimators': [30, 50, 100],
    'max_depth': [10, 11]
}

gs_gbmse = GridSearchCV(estimator=RandomForestMSE(n_estimators=10), param_grid=grid, 
    scoring='neg_mean_squared_error', n_jobs=-1)
# GradientBoostingMSE - не библиотечный класс, но он удовлетворяет scikit API
gs_gbmse.fit(X_train, y_train)
gs_gbmse.best_params_

{'max_depth': 11, 'n_estimators': 100}

In [11]:
predictor = RandomForestMSE(n_estimators=500, max_depth=21)

predictor.fit(X_train, y_train)

mean_squared_error(predictor.predict(X_test), y_test), mean_squared_error(np.ones_like(y_test) * np.mean(y_train), y_test)

(0.03468253997048785, 0.08147792929920478)

In [12]:
mean_squared_error(predictor.predict(X_test), y_test, squared=False), mean_squared_error(np.ones_like(y_test) * np.mean(y_train), y_test, squared=False)

(0.18623248903047998, 0.285443390708569)

In [13]:
for a, b in zip(predictor.predict(X_test[100:]), y_test[100:]):
    print(a, b)

0.01728655035589199 0
0.01645414248108129 0
0.016532742719227857 0
0.09483801856338911 0
0.01645414248108129 0
0.023182794064321027 0
0.10730331043804249 0
0.01645414248108129 0
0.06145383681521476 0
0.045755035099747786 0
0.01645414248108129 0
0.01767682558360593 0
0.01645414248108129 0
0.01645414248108129 0
0.01645414248108129 0
0.01645414248108129 0
0.01645414248108129 0
0.03012412356786907 0
0.01645414248108129 0
0.8965893027609717 1
0.02973640839662362 0
0.01645414248108129 0
0.068658066430302 0
0.01645414248108129 0
0.10188906481780624 0
0.01645414248108129 0
0.01645414248108129 0
0.01842376273424584 0
0.28262903663819394 0
0.03518279406432102 0
0.13178170718615173 0
0.05241049668544409 1
0.01645414248108129 0
0.027182794064321007 0
0.01645414248108129 0
0.01645414248108129 0
0.01645414248108129 0
0.0164560149369836 0
0.08250840776252719 0
0.25767566010485643 0
0.0164560149369836 0
0.020875735762909582 0
0.01645414248108129 0
0.01645414248108129 0
0.038675182142968355 0
0.2045638

Делаем вывод, что с определением статей на экономическую тематику (последний столбец отвечал за тему "Экономика") алгоритм обучения справляется (неидеально, но всё же лучше, чем константный прогноз). Можем сохранять.

In [14]:
from scipy import sparse

X = sparse.csr_matrix(np.vstack((X_train.toarray(), X_test.toarray())))
n_topics = len(column_to_category)
column_to_category, n_topics

(['69-я параллель',
  'Бывший СССР',
  'Дом',
  'Из жизни',
  'Интернет и СМИ',
  'Культура',
  'Мир',
  'Наука и техника',
  'Нацпроекты',
  'Путешествия',
  'Россия',
  'Силовые структуры',
  'Спорт',
  'Ценности',
  'Экономика'],
 15)

In [15]:
list_of_models: list = [None] * n_topics


for i in range(n_topics):
    gb = RandomForestMSE(n_estimators=5000, max_depth=20)
    gb.fit(X, y[:, i])
    list_of_models[i] = gb
    print(i + 1, '/', n_topics)

1 / 15
2 / 15
3 / 15
4 / 15
5 / 15
6 / 15
7 / 15
8 / 15
9 / 15
10 / 15
11 / 15
12 / 15
13 / 15
14 / 15
15 / 15


In [19]:
list_of_small_models: list = [None] * n_topics

for i in range(n_topics):
    gb = RandomForestMSE(n_estimators=100, max_depth=25)
    gb.fit(X, y[:, i])
    list_of_small_models[i] = gb
    print(i + 1, '/', n_topics)

1 / 15
2 / 15
3 / 15
4 / 15
5 / 15
6 / 15
7 / 15
8 / 15
9 / 15
10 / 15
11 / 15
12 / 15
13 / 15
14 / 15
15 / 15


In [20]:
list(zip(list_of_small_models[2].predict(X_test[:20]), y[4000:4020, 2]))

[(0.004391720155301317, 0),
 (0.006389394843817993, 0),
 (0.9106939759973913, 1),
 (0.006227839656548863, 0),
 (0.006389394843817993, 0),
 (0.00696848780591594, 0),
 (0.006389394843817993, 0),
 (0.00696848780591594, 0),
 (0.006308656666194005, 0),
 (0.006389394843817993, 0),
 (0.006389394843817993, 0),
 (0.006389394843817993, 0),
 (0.006389394843817993, 0),
 (0.8070376413806143, 1),
 (0.006389394843817993, 0),
 (0.006389394843817993, 0),
 (0.5431434905867817, 1),
 (0.006389394843817993, 0),
 (0.006389394843817993, 0),
 (0.006389394843817993, 0)]

In [17]:
file_model = open('model.pkl', 'wb')
pickle.dump(
    (basic_vectorizer, # преобразователь текста в вектор
    column_to_category, # преобразователь номера лучшей модели в тему
    list_of_models), # список моделей
    file_model)

In [21]:
file_model_small = open('model_small.pkl', 'wb')
pickle.dump(
    (basic_vectorizer, # преобразователь текста в вектор
    column_to_category, # преобразователь номера лучшей модели в тему
    list_of_small_models), # список моделей
    file_model_small)

In [22]:
from typing import List

file = open('model.pkl', 'rb')
data = pickle.load(file)

vectorizer, list_of_topics, list_of_models = data

vectorizer: CountVectorizer
list_of_topics: List[str]
list_of_models: List[RandomForestMSE]

def guess(text: str):
    stem_rus = RussianStemmer(False)

    out = ''
    for word in re.sub('[^{0-9а-яА-Я}]', ' ', text).lower().split(' '):
        out += stem_rus.stem(word) + ' '

    X_from_user = vectorizer.transform([out]).toarray()
    print()
    for gb, topic in zip(list_of_models, list_of_topics):
        print(gb.predict(X_from_user), topic)
    print()

In [23]:
text_economics = '''
ЦБ резко повысил ключевую ставку
ЦБ России повысил ключевую ставку до 9,5% годовых
Центробанк резко повысил ключевую ставку на один процентный пункт — до 9,5 процента годовых, говорится в сообщении на сайте регулятора. Решение прогнозировалось аналитиками и участниками долгового рынка.

Повышение стало восьмым подряд с марта прошлого года, когда ЦБ на фоне пандемии коронавируса и вызванного ею всплеска инфляции решил поднять ставку с рекордно низких 4,25 процента до 4,5 процента годовых. Предыдущее повышение состоялось 17 декабря и также составило один процентный пункт — с 7,5 до 8,5 процента.

В декабре ЦБ фиксировал в российской экономике признаки перегрева: восстанавливающийся спрос опережал по темпам роста предложение и возможности наращивания выпуска. Также свою роль сыграли растущие инфляционные ожидания населения, традиционно считающиеся надежным индикатором скорого повышения цен.

В качестве обоснования нынешнего решения руководства регулятора приводит слишком высокие темпы инфляции, опережающие октябрьские прогнозные значения ЦБ. «Проинфляционной остается конъюнктура мировых товарных рынков. Инфляционные ожидания пока не снижаются, оставаясь на многолетних максимумах. В этих условиях баланс рисков для инфляции еще больше сместился в сторону проинфляционных», — говорится в сообщении.

При этом ЦБ не исключил дальнейшего повышения ставки на следующих заседаниях. «По прогнозу Банка России, с учетом проводимой денежно-кредитной политики годовая инфляция снизится до 5–6 процентов в 2022 году и вернется к цели в середине 2023 года. В дальнейшем годовая инфляция будет находиться вблизи четырех процентов», — говорится в релизе.

Также ЦБ обращает внимание на зафиксированный в последнее время рост доходности облигаций федерального займа (ОФЗ), что свидетельствует о готовности участников рынка к дальнейшему повышению ставки, поскольку она выступает ориентиром (бенчмарком) для долговых инструментов на рублевом рынке в целом.

«Продолжается увеличение кредитных и депозитных ставок. Вместе с тем рост номинальных процентных ставок пока не обеспечил достаточного увеличения склонности к сбережению и не привел к более сбалансированной динамике кредитования. Приток средств на срочные рублевые депозиты происходит умеренным темпом. Одновременно с этим сохраняется высокая активность на рынке как розничного, так и корпоративного кредитования», — отмечает ЦБ, указывая на риски для устойчивости курса рубля и финансовой системы России.

В преддверии заседания совета директоров регулятора аналитики прогнозировали повышение ставки на один процентный пункт, обосновывая свою позицию сохранением геополитической напряженности, недавним ослаблением курса рубля, а также ужесточением денежно-кредитной политики властями западных стран, прежде всего Федеральной резервной системой США и Европейским центробанком.

Такая политика ведет к сокращению привлекательности рублевых активов для инвесторов-нерезидентов с точки зрения использования стратегии carry trade, предполагающей вложения на средства, заимствованные в другой иностранной валюте.

Кроме того, на решение ЦБ, по мнению специалистов, не мог не повлиять продолжающийся рост инфляции. По итогам 2021 года она составила 8,4 процента — при целевом уровне Центробанка в четыре процента годовых.

Следующее заседание совета директоров ЦБ, на котором будет рассматриваться вопрос изменения ключевой ставки, состоится 18 марта.
'''

In [24]:
text_sport = '''
«Это команда мечты!» Российские фигуристы разгромили сборную США и выиграли золото Олимпиады в Пекине
Ангелина Никитина
13-18 минут

Российские фигуристы — чемпионы командного турнира зимних Олимпийских игр-2022! На пекинском льду Камила Валиева, Марк Кондратюк, Анастасия Мишина и Александр Галлямов, Виктория Синицина и Никита Кацалапов по итогам всех выступлений в упорной борьбе вырвали победу у главных соперников — американцев. Бронзовые награды завоевали японцы, а хозяева Игр стали только пятыми. В этом виде соревнований золотых медалей у сборной России не было восемь лет: последний раз отечественные фигуристы побеждали в 2014-м, а на прошлых Играх заняли второе место. Самые яркие моменты олимпийского командного турнира — в материале «Ленты.ру».
Непростой выбор

По регламенту командного турнира по фигурному катанию на Олимпиаде, в каждом виде от каждой страны выступает один участник: одиночница, одиночник, спортивная и танцевальная пары. При этом на произвольную программу каждая сборная может заменить спортсменов в любых двух видах.

По итогам прокатов фигуристам присуждаются очки: победитель получает десять, финишировавший вторым — девять, третьим — восемь и так далее. Потом очки суммируются, и формируется общий рейтинг команд. Побеждает та, в активе которой окажется больше всего очков.

Состав сборной России держали в тайне до последнего. Более и менее понятно было, что на короткую программу заявят тех, кто выиграл в текущем сезоне чемпионат Европы: Камилу Валиеву — в соревновании девушек, Марка Кондратюка — в мужском одиночном катании, Анастасию Мишину и Александра Галлямова — в турнире спортивных пар, Викторию Синицину и Никиту Кацалапова — в танцах. Ожидания подтвердились.

По заменам же в прессе рассматривали самые разные варианты. В соревновании спортивных пар теоретическими претендентами на участие называли Александру Бойкову и Дмитрия Козловского, в танцах — Александру Степанову и Ивана Букина и даже Диану Дэвис и Глеба Смолкина. В женском одиночном — произвольную прочили Александре Трусовой, в мужском — Андрею Мозалеву. Россия практически при любом из этих раскладов должна была быть фаворитом турнира. Некоторую опасность представляла только сборная США.

И хотя шансы у россиян на бумаге действительно были сильно выше, чем у соперников, без нервов все-таки не обошлось.
«Очень обидно, потому что все шло хорошо»

Первый соревновательный день, 4 февраля, начался с короткой программы у мужчин. Этот вид программы у отечественной команды — самый слабый, и особых иллюзий перед стартом никто не питал. В соперниках у Кондратюка были мировой рекордсмен американец Нэйтан Чен, серебряный призер Игр в Пхенчхане японец Шома Уно, призер чемпионатов мира китаец Цзинь Боян, и главной задачей россиянина, чей технический набор вряд ли пока позволяет бороться за первое место, было не дрогнуть в этой компании и сделать свое.

Ученик Светланы Соколовской, к удивлению и радости многих, справился: чисто исполнил все элементы и финишировал третьим вслед за Ченом и Уно. В России в последние сезоны от чистых прокатов в мужской одиночке, прямо скажем, отвыкли, но катание Кондратюка уже который старт подряд вселяет надежду на то, что не все потеряно.

Соревнования продолжились выступлениями танцевальных дуэтов и спортивных пар — здесь, наоборот, ожидания были самыми высокими. Но на деле российских болельщиков поджидало сразу три мини-разочарования

Во-первых, во время ритм-танца на ровном месте ошибся Кацалапов: бросил исполнять дорожку шагов, не сделав выкрюк с ребра. Для дуэта такого уровня — а россияне являются действующими чемпионами мира — это не просто помарка, а серьезный срыв. Он стоил первого места — подопечные Александра Жулина полтора балла проиграли американцам, призерам ЧМ 2018 и 2019 годов Мэдисон Хаббелл и Захари Дохонью.

Очень обидно, потому что вся программа прошла хорошо. Это просто случайность, из-за которой мы потеряли много баллов. Теперь будем спокойнее. Нужно все держать под контролем

Во-вторых, в соревновании спортивных пар Мишина и Галлямов остановились менее чем в трех десятых от лидирующей позиции. Они исполнили программу с единственной помаркой — рассинхроном на параллельном вращении. Выиграл более опытный китайский дуэт, двукратные победители ЧМ Суй Вэньцзин и Хань Цун, причем превзошли соперников они по второй оценке, а технически ученики Тамары Москвиной были сильнее.

Это породило массу споров о предвзятости судейства и привилегиях для хозяев соревнований. Особенно обсуждения активизировались, когда выяснилось, что судья из США поставила спортсменам из России аномально низкий балл. Китайцев она оценила суммой на 3,7 балла больше, а у других судей перевес в их пользу не превысил 0,8 балла. Однако по факту это ничего не решало, потому что самые низкие и самые высокие оценки при финальном подсчете все равно отсекаются. Американская пара Алекса Книрим и Брэндон Фрейзер заняла третье место.

Как парники мы уже все себе доказали. И миру нам уже доказывать нечего. На этой Олимпиаде мы просто стараемся получить удовольствие, прочувствовать дух соревнований. Но, конечно, для нас большая честь — победить таких сильных спортсменов, особенно россиян. Саша и Настя — потрясающая пара с прекрасным катанием

В-третьих, все это привело к тому, что сборная России в общем зачете проиграла команде США два очка (26 против 28). Это отставание было некритичным, но и не слишком приятным — особенно в контексте того, что потеря как раз двух баллов вышла неожиданной и необязательной. На третьем месте расположилась команда Китая с 21 очком. Дальнейшие расклады зависели во многом от замен, которые сделают команды на произвольные программы.
«Очевидно, россиянки делают потрясающие вещи»

Второй день соревнований, 6 февраля, открывала короткая программа девушек-одиночниц. Определяющими здесь были два момента: справится ли с нервами Валиева и высоко ли поднимется в финальной таблице американка Карен Чен. Последняя, кстати, накануне выступлений обмолвилась, что готовится побороться с русской девочкой. Однако против любой из учениц Этери Тутберидзе, приехавших в Пекин, у Чен нет и никогда не было шансов, и заявление выглядело довольно странным.

Валиева была безупречна: она победила, исполнив программу с тройным акселем и по сумме превзойдя гроссмейстерские 90 баллов. Фигуристка установила новый олимпийский рекорд и почти вплотную приблизилась к собственному мировому — хороший знак в преддверии личного турнира, ведь это значит, что судьи не собираются скупиться ей на оценки. После проката россиянка признала, что была напряжена и чувствовала особенную ответственность, поэтому рада, что все получилось. Второе место заняла японка Вакаба Хигучи, третье — Маделин Скиза из Канады.

А вот Чен упала с тройного риттбергера, откатилась аж на пятую строчку, и этот результат фактически выключил сборную США из борьбы за золото. После выхода со льда американка призналась в неспособности повторить элементы, которыми владеют соперницы из России.

Очевидно, россиянки делают потрясающие вещи. Я могу только мечтать о таком, мне такое недоступно. Но я не могу думать о том, насколько хороша их команда. Мне надо просто концентрироваться на том, чтобы все вокруг меня были счастливы

Карен Ченамериканская фигуристка

В произвольной программе одиночников американцы заменили Чена на Винсента Чжоу, японцы заявили Юму Кагияму вместо Уно, а от России вновь выступал Кондратюк. Здесь отечественный фигурист имел куда больше шансов, чем в короткой, и не растерялся. Он чисто исполнил программу (хотя первый же прыжок, четверной тулуп, еле спас, на одном из каскадов с большим трудом миновал бортик, а в конце сделал только двойной сальхов вместо тройного) и стал вторым. Победил Кагияма, Чжоу замкнул тройку.

Успех Кондратюка вызвал бурю оваций. Пользователи русскоязычного Twitter вывели на первое место в трендах хештег #МыРусскиеСНамиМарк. Хореограф Илья Авербух в комментарии «Спорт-Экспресс» заметил, что лучше выступить россиянин просто не мог бы. А американский актер Бен Стиллер заявил: «Респект тому российскому парню, который отжег под Jesus Christ Superstar» (музыкальное сопровождение в произвольной программе Кондратюка — музыка из этой рок-оперы — прим. «Ленты.ру»).

В результате сборная России наконец захватила лидерство в общем зачете: отечественные спортсмены отыграли разрыв от американцев и получили преимущество в три балла (45 против 42). На третью строчку поднялись японцы (39).
«Синяки будут, но мы их замажем»

Накануне заключительного дня соревнований Федерация фигурного катания на коньках России (ФФККР) объявила, что замен на оставшиеся программы командного турнира в сборной не планируется. Решение неожиданное: с учетом оставшихся видов золоту команды уже ничего не угрожало всерьез, так почему бы не дать еще двум участникам получить статус олимпийских чемпионов? Да и поберечь лидеров для личных стартов было бы логично. Сборная России между тем стала единственной командой на турнире, обошедшейся одним составом на обе программы.

Последний день соревнований открывала произвольная спортивных пар. Мишина и Галлямов, которые из-за поражения в короткой выходили на лед перед китайцами, больших проблем теперь не испытали и выиграли с большим отрывом. Лучший балл спортсмены получили, несмотря на страшное с виду падение, которое случилось на выезде с поддержки. По словам спортсменов, причина может быть в раннем подъеме перед стартом.

Я держала лицо. Все сделали, но в последние пять секунд прилегли. Заснули. Сильно ничего не болит. Синяки будут, но мы их замажем. Самые большие синяки — от недосыпа
Китайцы сделали замену: вместо двукратных чемпионов мира теперь катались менее титулованные Чэн Пэн и Ян Цзинь, и у них прокат вышел не самым удачным. В итоге хозяева турнира остановились на третьей строчке. Без замен обошлись американцы, от которых выступали Книрим и Фрейзер, однако у них выступление также не задалось: масса мелких срывов привела их на последнее место в итоговой таблице.

Второе место досталось японцам Рику Миура и Рюичи Кихара, и это добавило интриги в борьбе за серебро. Перед двумя заключительными видами Япония сравнялась с США (команды набрали по 48 очков).

В активе россиян же стало 55 очков. Это была по факту досрочная победа

Так что Синицина и Кацалапов на лед выходили в максимальном спокойствии. Командная победа была в кармане, но вопросом принципа было выиграть после ошибки в ритм-танце. Не вышло и в этот раз: россияне получили балл штрафа за передержанную поддержку, и он оказался решающим. Американцы Мэдисон Чок и Эван Бейтс опередили подопечных Жулина на девять десятых. На публике российский дуэт не придал очередному локальному проигрышу большого значения, однако... За их результаты в грядущем личном первенстве становится все тревожнее. Японцы в свою очередь проиграли китайцам и позволили сборной США вновь выйти вперед в общем рейтинге.

Я пока даже не осознал все происходящее, не осознал, что мы уже чемпионы. Нужно дать выступить Камиле Валиевой, поболеть за нее от всей души. И тогда будем радоваться тому, что победили

Валиева исполнила свое «Болеро» с двумя четверными (сальхов и тулуп в каскаде с тройным) и тройным акселем. На приземлении с третьего квада, тулупа, спортсменка упала, однако это, понятно, не помешало ей победить с космическим отрывом. Камила осталась очень недовольна прокатом — радость от победы в команднике пришла к ней только в kiss-and-cry. Между тем эта ученица Тутберидзе стала первой фигуристкой в истории, исполнившей четверной прыжок на Играх. Вторая строчка осталась за японкой Каори Сакамото, третье — за канадкой Скизой. Результат Сакамото не повлиял на финальный расклад: японцы остались с бронзой, американцы удержали серебро.

Это была детская мечта, я с трех лет говорила маме, что стану олимпийской чемпионкой. И вот, мечта сбылась. Теперь я буду еще работать, чтобы сбылась еще одна мечта

Камила Валиевароссийская фигуристка
«Эта победа должна вдохновить их»

Теперь эмоции можно было не сдерживать. Фигуристы принесли в копилку сборной России второе золото пекинской Олимпиады.

Одним из первых победу прокомментировал Александр Жулин. «Нет слов — просто потрясающие эмоции, неописуемые! Конечно, мы очень рады командному золоту, это было важно. Сегодня у нас была настоящая команда мечты, по-другому ее не назовешь. Все ребята замечательные, выкладывались на сто процентов и забрали заслуженное золото», — цитирует его Sport24.

Хореограф Илья Авербух отметил, что все россияне откатались ровно, без фатальных ошибок. «Сегодня ребята просто взяли свое: то, что и должны были взять. Все понимали, что сегодня победитель определится чисто технически. Ребята доказали, что сборная России — сильнейшая в мире фигурного катания», — сказал он в комментарии «Чемпионат.com».

Мы были фаворитами этого турнира, но быть фаворитом и выигрывать — это разные вещи. Сегодня доказали, что российская команда — самая сильная в мире

А олимпийская чемпионка 2018 года Алина Загитова в эфире Первого канала заявила, что немного завидует новоиспеченным победителям. «Думаю, что мы выиграли еще вчера. Но, конечно, каждый балл был очень важен. Мне очень интересно, каково Камиле Валиевой было выходить, зная, что они уже победили. Можно было реально ничего не делать. Эта победа должна вдохновить их выступить в личном турнире еще лучше», — подчеркнула она.
***

Вокруг целесообразности организации командных турниров по фигурному катанию на Олимпиаде продолжаются споры (ведь порой статус призеров получают спортсмены, которые в личном первенстве никогда не были даже в топ-5 чемпионата мира, в этот раз такой стала американская спортивная пара Книрим и Фрейзер), но одно можно сказать точно: такие соревнования — это зрелищно и интересно.

Командный турнир в Пекине стал третьим в истории Олимпийских игр: дебютировал этот вид соревнований в 2014 году в Сочи. Тогда отечественная сборная победила, и это стало знаковым событием, подтверждающим, что российская школа — главная в мировом фигурном катании. Но спустя четыре года национальная команда осталась с серебром, уступив канадцам.

Впереди у фигуристов борьба в личном первенстве. Отечественные спортсмены претендуют не на одну победу и точно не раз окажутся в призах, но все это — чуть позже. Сейчас же — самое время насладиться моментом. Принципиально важное золото вернулось в Россию, нам снова нет равных!
'''

In [25]:
text_science = '''
Биотехнологии в медицине меняют мир. Российские ученые рассказали о разработках, меняющих подходы в лечении ряда болезней

Мечты о том, чтобы вырастить жизнеспособные органы для человека, внедрить в организм датчики здоровья или обойтись таблеткой там, где раньше требовался целый протокол лечения, похоже, становятся реальностью. Что сегодня происходит в биомедицине, как быстрее вывести из научных центров «на улицы» передовые разработки и какие проблемы нужно решить российской науке, чтобы выйти на ведущие позиции в мире, — в материале «Ленты.ру».
Секвенирование генома — тихая революция

Стремительное развитие технологий привело к внедрению новых методов и средств охраны и восстановления здоровья людей, кардинально меняющих подходы в медицине. Об этом в ходе пресс-конференции «Россия и мир: достижения науки в области биомедицины», состоявшейся в пресс-центре «Россия сегодня», рассказали ведущие эксперты.

Участие в круглом столе приняли академик РАН, президент ФГБУ «Национальный медицинский исследовательский центр детской гематологии, онкологии и иммунологии имени Дмитрия Рогачева» Минздрава России Александр Григорьевич Румянцев, вице-президент РАН, руководитель отдела фундаментальной и прикладной нейробиологии Государственного научного центра социальной и судебной психиатрии имени В.П. Сербского, заведующий кафедрой медицинских нанобиотехнологий Российского государственного медицинского университета Владимир Павлович Чехонин, доктор медицинских наук, профессор, вице-президент по внедрению новых медицинских технологий АО «ГЕНЕРИУМ» Дмитрий Анатольевич Кудлай, доктор медицинских наук, член-корреспондент РАН, директор ФГБНУ «МГНЦ», главный внештатный специалист по медицинской генетике Минздрава России Сергей Иванович Куцев, а также доктор биологических наук, профессор РАН, директор Департамента науки и инновационного развития здравоохранения Минздрава России Игорь Викторович Коробко.

За последние годы биомедицинская наука превратилась в основной двигатель трансформационных процессов в медицине. Форматы оказания медицинской помощи стали более персонализированными, а подходы специалистов сместились в сторону превентивной работы с пациентами. Влияние фундаментальных научных исследований на клиническую практику оказалось колоссальным. Революцией, незаметной для пациентов, но очевидной для всей медицинской общественности, стало секвенирование генома, которое способно выявить почти все изменения в ДНК пациента. До этого момента многие дефекты в организме, реализуемые в жизни человека от зачатья до глубокой старости и приводящие к широкому кругу заболеваний, описывались посиндромно, без причины появления болезни, пояснил Александр Григорьевич Румянцев.

В качестве наглядного примера того, как это открытие буквально перевернуло методику лечения и спасло огромное число жизней, эксперт назвал ситуацию с хроническим миелолейкозом. Благодаря молекулярным исследованиям учеными была уточнена причина этого заболевания. Это позволило предложить новый необычный вид лечения, вмешивающийся непосредственно в генетическое расстройство. Если раньше абсолютно смертельное заболевание лечилось препаратами, облучением, специальными методами оперативного лечения, трансплантацией костного мозга, то новая терапия заложена в таблетку и приводит такого пациента к молекулярной ремиссии.

Еще один пример — острый лимфобластный лейкоз, на который приходится до 40 процентов всех онкологических заболеваний у детей. Тридцать лет назад, по словам А.Г. Румянцева, было лишь 7 процентов выздоровевших детей. Многолетняя работа нескольких коллективов позволила добиться того, чтобы в прошлом году этот знаменатель в России был доведен до 84 процентов. Протоколы лечения совершенствовались десятилетиями, к работе были привлечены специалисты из Белоруссии, Казахстана, Киргизии, Узбекистана, Армении. Однако изучение антител к детерминантам лимфоцитов фактически позволило добиться невозможного — лечения, благодаря которому удалось выйти на полную ремиссию у всех больных.
Своевременная диагностика

Использование достижений биомедицины позволяет не только лечить, но и предотвращать развитие многих заболеваний. Особенно это актуально при орфанных, то есть редких заболеваниях, в случае которых затруднительна постановка диагноза, а значит и начало своевременного лечения. В РФ к таким заболеваниям относят патологию, обнаруживаемую с частотой один случай на 10 тысяч населения. Однако не стоит недооценивать проблему: в совокупности орфанных болезней более чем 7 тысяч и 90 процентов из них являются наследственными, то есть связаны с изменением генетического аппарата пациентов. «Своевременная диагностика орфанных болезней — один из важных моментов в современной медицине, которая требует разработки различных диагностических подходов», — отметил Сергей Иванович Куцев.

По его мнению, лучшим подходом является неонатальное скрининговое обследование, которое проводится при помощи тандемной масс-спектрометрии. Оно проводится для всех новорожденных на ряд наследственных заболеваний, для которых существует эффективное лечение. Обнаруженное на ранних стадиях заболевание и вовремя начатое лечение способны сохранить жизнь ребенка и избежать его инвалидизации. Напомним, что с 2022 года программа неонатального скрининга на выявление редких заболеваний в России будет расширена с 5, как это было раньше, до 36 наследственных болезней. В расширенный перечень скрининговой программы войдёт также исследование TREC и KREC для диагностики первичных иммунодефицитных состояний.

Основные тренды в современной медицине связаны с необходимостью внедрения принципа персонализации при лечении конкретного пациента. Этому способствует увеличение объемов знаний о механизмах развития заболеваний. Это позволяет при одной и той же патологии на основании определенных признаков или биомаркеров разделить таких пациентов на группы, понимая, что каждой группе нужно свое специфическое лечение, рассказал Игорь Викторович Коробко. «Фактически мы переходим от существующей парадигмы «одно лекарство для всех» к подходу нужное лекарство для конкретного пациента в правильное время в нужной дозировке. Персонализация входит в рутинную практику», — отметил эксперт.

Новые достижения последних десятилетий в области биомедицины, открытия в области генных и клеточных технологий, появление тканевой инженерии меняют лицо современной медицины. При этом новые возможности, которые современная наука предоставляет врачам-практикам, позволяют рассчитывать на излечение, избежание инвалидизации и даже предотвращение развития ранее смертельных заболеваний.

«Трендами биомедицины на современном этапе являются молекулярная генетика и биология, биотехнология и биоинформатика. Это современные направления науки, которые формируют возможность создания так называемых научных биомедицинских платформ для лечения целого спектра социально значимых заболеваний», — отметил Владимир Павлович Чехонин. По его мнению, в условиях глобализации возникают проблемы, которые наука в отдельной стране решить не может, должна быть система, которая объединила бы усилия специалистов и учёных различных стран мира. Проблемы СПИДа, инсультов, сахарного диабета, целого ряда инфекционных, сердечно-сосудистых заболеваний не могут быть решены отдельными группами ученых.

Один из трендов, который может стать доминирующим в ближайшие годы, — появление инновационных генных и клеточных технологий. В России над масштабированием биомедицинских клеточных продуктов работает биотехнологическая компания «ГЕНЕРИУМ», которая предложила уникальную разработку для лечения повреждений хрящевых компонентов коленных суставов. «На смену весьма травматичному и дорогому эндопротезированию приходят новые технологии — хондросферы, когда мини-доступом клетки хрящевой ткани забираются из здорового участка поверхности сустава, далее в изоляторах наращивается масса новых. Затем конгломерат здоровых клеток возвращается обратно в сустав. В результате их прицельного оседания на поврежденных зонах удается полностью нивелировать до 8-10 квадратных сантиметров серьезных дефектов», — рассказал Дмитрий Анатольевич Кудлай. Компания «ГЕНЕРИУМ» получила первое в России разрешение на клинические испытания безопасности и эффективности трансплантации биомедицинского клеточного продукта – препарата для восстановления хрящевой ткани коленного сустава. Ожидается, что препарат позволит эффективно лечить коленные суставы, что в перспективе снизит потребность в операциях по эндопротезированию.

Несмотря на вдохновляющие результаты российской науки, существует спектр проблем, требующих решения. По словам Дмитрия Анатольевича Кудлая, Россия чувствует себя уверенно на мировой арене и остается одним из наиболее динамичных рынков с точки зрения перспектив развития. Так, например, Россия входит в ТОП-5 по потенциалу научных коллективов в области регенеративной медицины, фармацевтический рынок занимает те же позиции. Компания «ГЕНЕРИУМ» в случае с орфанной тематикой и молекулярной медициной попала в ТОП-7 по портфелю. «Однако, по-прежнему, значительные суммы госбюджета уходят на закупку дорогостоящих импортных препаратов. Безусловно, это было полезно на определенном этапе развития России, но эти деньги должны постепенно разворачиваться в сторону отечественных разработок», — отметил Кудлай.

Эксперты ожидают, что со стороны государства будет проведена работа по решению проблемы патентной защиты отечественных разработок и продвижению позиций российской науки в мире.

Барьером на пути развития медицины будущего также является отсутствие технологий, отмечает Александр Григорьевич Румянцев. «В нашей стране есть уникальные научные разработки, но путь до производства позиций, которые могут выйти на рынок (оборудование, аппаратура, лекарства, реактивы), невозможен без специальных технологий. Здесь наблюдается серьезное отставание, которое в некотором объеме позволяют компенсировать разработки Китая, с которым у нас хороший контакт. Но России нужно самой транслировать научные разработки в практику. Это позволит сделать научно обоснованный прорыв в области различных дисциплин, в том числе и в области медицины», — отметил эксперт.

Последнее невозможно без работы над законодательной базой, которая не успевает за развитием биотехнологий. Органы исполнительной власти ограничивают возможности врачей в использовании целого ряда разработок. В настоящее время Государственная дума и Министерство здравоохранения обсуждают несколько законодательных поправок в закон о клеточных технологиях, поскольку в РФ до сих пор нет понятия биобанкинга, отсутствует законодательная база в отношении федерального банка доноров. «Мы не можем вынести из научных центров "на улицу" целый ряд технологий, заниматься их тиражированием», — говорит А.Г. Румянцев.

Сергей Иванович Куцев также указывает на то, что научные центры в России должны быть мультиплицированы, тогда стране удастся реализовать огромный потенциал в области научных фундаментальных исследований.
'''

In [26]:
guess(text_economics)


[6.09672913e-05] 69-я параллель
[0.01338544] Бывший СССР
[0.14227384] Дом
[0.01204617] Из жизни
[0.02494945] Интернет и СМИ
[0.01103621] Культура
[0.06925589] Мир
[0.02257861] Наука и техника
[0.00432784] Нацпроекты
[0.00625016] Путешествия
[0.11782523] Россия
[0.00843191] Силовые структуры
[0.00724924] Спорт
[0.00396809] Ценности
[0.86215884] Экономика



In [27]:
guess(text_sport)


[0.00935871] 69-я параллель


In [ ]:
guess(text_science)


[0.03742907] 69-я параллель
[0.28177616] Бывший СССР
[0.05166044] Дом
[0.19228693] Из жизни
[0.27123657] Интернет и СМИ
[0.11493181] Культура
[0.12558802] Мир
[0.492366] Наука и техника
[0.13578511] Нацпроекты
[0.0083865] Путешествия
[0.50466484] Россия
[0.04818855] Силовые структуры
[0.01093587] Спорт
[0.00553196] Ценности
[0.54217813] Экономика



In [ ]:
text_internet = '''
Парень создал игру для любимой. Он отказался зарабатывать на приложении, но оно стало культовым и сделало его миллионером

Весной 2020 года американский разработчик британского происхождения Джош Уордл (Josh Wardle) создал онлайн-головоломку под названием Wordle. Изначально игра была предназначена только для двоих: самого программиста и его возлюбленной. Спустя полтора года, когда программист наконец решил показать свой проект общественности, игра начала стремительно завоевывать популярность: к началу 2022 года число ее пользователей превысило 10 миллионов. Уордл не планировал зарабатывать с помощью головоломки, однако в итоге продал и игру, и права на нее газете The New York Times (NYT) более чем за миллион долларов. Как обычному программисту удалось это сделать и почему Wordle стала культовой игрой — в материале «Ленты.ру».
«Цели заработать не стояло никогда»

Игра Wordle похожа на многие другие приложения: в ней пользователю нужно отгадать слово из пяти букв. В то же время она во многом отличается от них. Во-первых, игра совершенно не пытается заставить человека тратить на нее все свободное время: доступ к ней открывается лишь один раз в сутки. Многие приложения регулярно отправляют навязчивые уведомления, снова и снова предлагая поиграть еще, а Wordle держит дистанцию с игроками. «Это то, на что вы потратите всего три минуты в день. И все. Игра не хочет отнимать у вас больше времени», — заверял Уордл.

Сам разработчик считает, что это ограничение стало одной из причин феноменальной популярности приложения и спровоцировало интерес к головоломке. По мнению программиста, возможность играть только один раз в день вызывает у пользователей чувство дефицита, что в итоге обостряет желание снова вернуться к Wordle. При этом Уордл неоднократно признавался, что частично перенял эту особенность у онлайн-игры Spelling Bee (в ней пользователям нужно правильно писать слова) от The New York Times Games.

Еще одной непривычной особенностью игры стало полное отсутствие платных функций. Игра всегда была бесплатной, в ней никогда не было рекламы, а сам создатель приложения никогда не преследовал цели заработать с помощью Wordle. Более того, он сам ежемесячно платил по 100 долларов, чтобы поддерживать работу игры. «На самом деле цели заработать деньги не стояло никогда», — рассказывал Уордл.

Когда игра стала стремительно набирать популярность, ее создателю начали советовать запустить платную премиум-версию или добавить рекламу. От обоих предложений программист настойчиво отказывался. «Я не знаю, может быть, я идиот. Ничего из этого мне не нравится. Я думаю, что, поскольку мое намерение изначально было таким, мне было легко сказать нет монетизации. Если бы я пытался сделать вирусную игру, я бы наверняка вел себя по-другому», — признавался разработчик игры.
«Захватила коллективное воображение»

Уордл запустил Wordle на своем сайте powerlanguage.co.uk в октябре 2021 года. Еще в ноябре о бесплатной игре, в которой нужно отгадывать слова, знало всего 90 человек, спустя два месяца аудитория приложения насчитывала около 300 тысяч пользователей, а в январе 2022 года это число превысило 10 миллионов.

31 января этого года права на популярную онлайн-головоломку Wordle приобрела газета The New York Times, которая заплатила Джошу Уордлу «семизначную сумму». Точная прибыль от сделки не разглашается, однако очевидно, что программист заработал на игре не менее одного миллиона долларов, сообщает издание Time.

Игрокам разработчик обещал, что в первое время Wordle останется бесплатной для всех пользователей, но не исключил, что в будущем газета сделает доступ к головоломке платным. Уордл добавил, что игру перенесут на платформу The New York Times, и пообещал, что постарается сделать так, чтобы данные и достижения игроков сохранились. Кроме того, он заверил, что в сам игровой процесс не внесут никаких изменений.

Если вы похожи на меня, вы, вероятно, просыпаетесь каждое утро, думая о Wordle и наслаждаясь этими драгоценными моментами открытий, удивления и достижений. Игра сделала то, что сделали так мало игр: она захватила наше коллективное воображение и немного сблизила всех нас
Джонатан Найт генеральный менеджер The New York Times Games

Уордл назвал продажу головоломки естественным шагом и добавил, что ему близок подход The New York Times Games к играм. «Я давно восхищался подходом издания к качеству их игр и уважением, с которым они относятся к своим игрокам. Их ценности совпадают с моими в этих вопросах, и я в восторге от того, что они будут распорядителями игры, которая движется вперед», — рассказал программист после продажи приложения.

Генеральный менеджер The New York Times Games Джонатан Найт (Jonathan Knight) отметил: «Мы очень рады стать новым домом и гордыми распорядителями этой волшебной игры, и для нас большая честь помочь донести заветное творение Джоша Уордла до большего количества игроков в ближайшие месяцы». Так Найт анонсировал присоединение Wordle к The New York Times Games.

Издание выкупило онлайн-головоломку на пике ее популярности не случайно: с ее помощью The New York Times планирует привлекать аудиторию на свою платформу. Вполне вероятно, в будущем доступ к Wordle станет платным, как и материалы New York Times, в том числе игры. За участие в игровом сообществе The New York Times Games, которое насчитывает более одного миллиона пользователей, подписчики издания платят пять долларов в месяц или 40 долларов в год.

Издание уже несколько лет активно инвестирует в онлайн-головоломки. В 2020 году игры от The New York Times пользовались колоссальной популярностью, в результате чего акции компании выросли на 61 процент по итогам года. В 2021 году самыми популярными ключевыми словами на сайте газеты оказались не «Трамп», «Байден» или даже «коронавирус», а «кроссворды» и Spelling Bee. В будущем NYT планирует только наращивать инвестиции в онлайн-игры.

Самое яркое чувство, которое я сейчас испытываю, это не радость. Это облегчение

Популярность по ошибке

Внезапный успех игры Уордла привел и к некоторым курьезам. Пять лет назад приложение для AppStore с похожим названием — Wordle! — выпустил программист Стивен Кравотта (Steven Cravotta). В итоге популярность онлайн-головоломки Уордла привела к тому, что число скачиваний игры Кравотты резко выросло. С начала запуска до истории с Wordle это приложение скачали около 100 тысяч пользователей. Со временем Кравотта перестал обновлять игру, и интерес к ней полностью угас. Однако после того, как об игре Уордла стали говорить журналисты таких крупных изданий, как The New York Times, Wall Street Journal, а также комик и телеведущий Джимми Фэллон, приложение Кравотты по ошибке загрузило более 200 тысяч человек.

Кравотта заметил рост скачиваний, но предположил, что кто-то случайно направил искусственную накрутку. Позже разработчик понял, что загрузки реальные: люди просто путают его приложение с нашумевшей головоломкой. «Крупные издания публиковали статьи о Wordle, но ни одно из них четко не объяснило, что речь шла о браузерной игре. Естественно, люди отправлялись в AppStore, чтобы скачать приложение», — пояснил он.

В итоге Кравотта связался с Джошем Уордлом и рассказал ему о том, как популярность Wordle повлияла на его собственное приложение. В итоге разработчики решили, что пожертвуют деньги, которые Кравотта заработал случайно, в образовательную организацию Boost Oakland. «Очень рад поддержать такую удивительную программу, которая занимается повышением грамотности молодежи. Мы чувствуем, что деньги помогут изменить ситуацию», — написал Кравотта. Информацию также подтвердил и сам Уордл.
«Игра, которая понравится ей»

Джош Уордл вырос на юге Уэльса в семье фермеров, получил высшее образование в Лондоне, а позже поступил в Орегонский университет, где получил степень магистра в области цифрового искусства. После учебы он устроился на работу в Кремниевой долине и стал инженером-программистом на популярной интернет-платформе Reddit. Для сайта он создал два успешных проекта The Button (2015) и Place (2017).

В отличие от двух других громких проектов Уордла, игру Wordle он создал в одиночку. Он придумал прототип головоломки еще в 2013 году, но отказался от идеи после того, как ее раскритиковали друзья. Спустя семь лет, в разгар пандемии, разработчик подарил игре новую жизнь и все-таки запустил приложение, но предназначено оно было только для возлюбленной программиста Палак Шах (Palak Shah). На карантине в 2020 году девушка стала регулярно играть в Spelling Bee, поэтому Уордл решил придумать игру, которая бы понравилась его девушке. Хотя игра в итоге стала доступна многомиллионной публике, Палак Шах признавалась: она очень ценит то, что Уордл изначально создавал Wordle исключительно для нее в знак проявления любви. 
'''

In [ ]:
text_culture = '''
Стало известно место похорон актрисы Зинаиды Кириенко
Народную артистку РСФСР Зинаиду Кириенко похоронят на Троекуровском кладбище

Народная артистка РСФСР Зинаида Кириенко будет похоронена во вторник, 15 февраля, на Троекуровском кладбище. Об этом сообщает РИА Новости со ссылкой на Союз кинематографистов России.

Церемония прощания с актрисой пройдет в Большом зале Дома кино в 11:00 по московскому времени.

О кончине Зинаиды Кириенко стало известно в субботу, 12 февраля. Артистка, ранее госпитализированная с коронавирусом, умерла на 89-м году жизни.

11 февраля сообщалось, что у актрисы был обнаружен рак поджелудочной железы и геморрагический инсульт. Она была частично парализована и находилась в реанимационном отделении. До этого она также получила положительный результат теста на COVID-19.

Зинаида Кириенко известна по таким фильмам и сериалам, как «Судьба человека», «Тихий Дон», «Любовь земная», «Два капитана», «Письма к Эльзе», «Любить по-русски 2», «Казаки», «Вдали от Родины», «Господин Великий Новгород» и «Самый медленный поезд».
'''

In [ ]:
text_travel = '''
Бортпроводник раскрыл самые отвратительные просьбы пассажиров на борту самолета
Бортпроводник рассказал, что один пассажир попросил смешать ему кока-колу и молоко

Бортпроводник зарубежных авиалиний раскрыл самые нелепые просьбы пассажиров на борту за всю свою карьеру. Об этом он рассказал изданию Fodor's Travel.

По словам сотрудника авиакомпании, за десять лет работы он слышал много странных просьб от пассажиров, которые вызывали у него отвращение. Например, один турист захотел коктейль из кока-колы и молока — как утверждал мужчина, такой напиток позволяет ему избавляться от похмелья. Для этого он налил газировку на треть в стакан и добавил немного молока. По его словам, вкус напоминал мороженое.
Материалы по теме:
«Мы находимся в группе максимального риска»Российские бортпроводники — о работе во время коронавируса, нервных пассажирах и страхе
20 апреля
«Представьте, какой эффект на пассажиров окажут девушки»Мини-юбки, молодость и сексуальность. Как менялись требования к стюардессам в разные времена
9 июня

Другой пассажир попросил смешать ему в одном стакане имбирный эль, минеральную воду Selters, порцию клюквенного и апельсинового сока, немного спрайта, лимон и лайм. Как признался бортпроводник, ему стало интересно, случайно ли турист придумал такой рецепт и специально не добавил сладкой газировки, однако, к его удивлению, мужчина заметил недостающий ингредиент.

Еще один пассажир попросил пять упаковок печенья и половинку лайма: мужчина принялся выдавливать сок на закуску, таким образом имитируя вкус лаймового пирога. Как заметил бортпроводник, сходство с десертом действительно было.

Ранее в январе члены экипажа вспомнили самые нелепые случаи с пассажирами за свою карьеру. Так, бортпроводница рассказала историю, как одна женщина весь полет ходила по проходу и убеждала всех вокруг, что у нее клаустрофобия. Помимо прочего, она требовала бесплатные алкогольные напитки и была глубоко возмущена, когда ей отказали в шестой добавке.
'''

In [ ]:
guess(text_internet)


[0.00028939] 69-я параллель
[0.02498148] Бывший СССР
[0.01562173] Дом
[0.08396674] Из жизни
[0.50094974] Интернет и СМИ
[0.02097286] Культура
[0.22502565] Мир
[0.33063079] Наука и техника
[0.01480784] Нацпроекты
[0.15349674] Путешествия
[0.22343522] Россия
[0.02235725] Силовые структуры
[0.06172339] Спорт
[0.07658118] Ценности
[0.23141143] Экономика



In [ ]:
guess(text_culture)


[2.19828793e-05] 69-я параллель
[0.0140194] Бывший СССР
[0.00796237] Дом
[0.01257342] Из жизни
[0.01654028] Интернет и СМИ
[0.5816954] Культура
[0.04630188] Мир
[0.01185933] Наука и техника
[6.06931835e-06] Нацпроекты
[0.00610332] Путешествия
[0.13519015] Россия
[0.00743223] Силовые структуры
[0.00729557] Спорт
[0.00392887] Ценности
[0.01683853] Экономика



In [ ]:
guess(text_travel)


[2.12494773e-05] 69-я параллель
[0.01769475] Бывший СССР
[0.00770118] Дом
[0.07275122] Из жизни
[0.02960653] Интернет и СМИ
[0.01014657] Культура
[0.06908888] Мир
[0.01636478] Наука и техника
[6.06931835e-06] Нацпроекты
[0.81650854] Путешествия
[0.06845746] Россия
[0.00973975] Силовые структуры
[0.12430295] Спорт
[0.04898746] Ценности
[0.01898974] Экономика

